In [2]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import torch
import os
from PIL import Image
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F 
import pickle

In [3]:
import torchvision.models as models
model = models.resnet34(pretrained=True)
model.fc = nn.Sequential(nn.Linear(512, 121))


In [4]:
# Setup: initialize the hyperparameters/variables
# Setup: initialize the hyperparameters/variables
num_epochs = 10           # Number of full passes through the dataset
batch_size = 64         # Number of samples in each minibatch
learning_rate = 0.01  
seed = np.random.seed(0) # Seed the random number generator for reproducibility
p_val = 0.1              # Percent of the overall dataset to reserve for validation
p_test = 0.2             # Percent of the overall dataset to reserve for testing


#TODO: Convert to Tensor - you can later add other transformations, such as Scaling here

transform = transforms.Compose([
        #transforms.RandomResizedCrop(224),
        transforms.ToPILImage('L'),
        transforms.Resize([224,224],interpolation=2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])


In [5]:
# Check if your system supports CUDA
use_cuda = torch.cuda.is_available()

# Setup GPU optimization if CUDA is supported
if use_cuda:
    computing_device = torch.device("cuda")
    extras = {"num_workers": 1, "pin_memory": True}
    print("CUDA is supported")
else: # Otherwise, train on the CPU
    computing_device = torch.device("cpu")
    extras = False
    print("CUDA NOT supported")

model = model
model = model.to(computing_device)
print("Model on CUDA?", next(model.parameters()).is_cuda)    

CUDA is supported
Model on CUDA? True


In [6]:
### need to modify
# Setup the training, validation, and testing dataloaders
from plankton_dataloader import create_split_loaders
root_dir = "../dataset/data_subset/"
train_loader, val_loader, test_loader = create_split_loaders(root_dir,batch_size, seed, transform=transform, 
                                                             p_val=p_val, p_test=p_test,
                                                             shuffle=True, show_sample=False, 
                                                             extras=extras)


### ADD Weights

In [ ]:
# pickle_in = open("../dataset/class_count_list.pickle","rb")
# class_count_list = pickle.load(pickle_in)
# total_count = sum(class_count_list)
# weights = [total_count / item for item in class_count_list] #[ 1 / number of instances for each class]
# class_weights = torch.FloatTensor(weights).cuda()
# criterion = nn.CrossEntropyLoss(weight = class_weights).to(computing_device)

In [ ]:
# sum(class_count_list)

In [7]:
criterion = nn.CrossEntropyLoss().to(computing_device)

optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0001)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [8]:
def validate(val_loader,model,optimizer):
    start = time.time()
    sum_loss = 0.0
    sum_accuracy = 0.0
    list_sum_loss = []
    num = 0
    for mb_count, (val_images, val_labels) in enumerate(val_loader, 0):
        model.eval()
        with torch.no_grad():  
            optimizer.zero_grad()      
            val_images = torch.squeeze(torch.stack([val_images,val_images,val_images], dim=1, out=None))
            val_images, val_labels = val_images.to(computing_device), val_labels.to(computing_device)
            val_labels = val_labels.type(torch.cuda.FloatTensor)
            outputs = model(val_images)
            loss = criterion(outputs,torch.max(val_labels, 1)[1])
            sum_loss += loss
            output_np = outputs.cpu().detach().numpy()
            label_np = val_labels.cpu().detach().numpy()

            accuracy_train = accuracy(label_np, output_np)
            sum_accuracy += accuracy_train
            #print('validation accuracy',accuracy_train)
    print("Vali Loss: {}, Vali Accuracy: {}".format(1.0 * sum_loss / mb_count, sum_accuracy / mb_count))
    print("validation time = ", time.time()-start)    
    return (1.0*sum_loss / mb_count).item(), sum_accuracy / mb_count

In [9]:
def accuracy(out, labels):
    outputs = np.argmax(out, axis=1)
    lab = np.argmax(labels, axis=1)
    return np.sum(outputs==lab)/float(lab.size)

In [10]:
check_point = torch.load('./checkpoint/5_model_epoch80.pth')
check_point['epoch']


6

In [13]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=10,resume=False, direct=""):
    since = time.time()
    total_loss = []
    if resume == False:
        avg_minibatch_loss = []
        avg_minibatch_accuracy = []
        total_vali_loss = []
        total_vali_accuracy = []
        epc_save = 0 # MAYBE
    else: 
        print('Resume model: %s' % direct)
        check_point = torch.load(direct)
        model.load_state_dict(check_point['state_dict'])
        optimizer.load_state_dict(check_point['optimizer'])
        avg_minibatch_loss = list(np.load('avg_train_loss.npy'))
        avg_minibatch_accuracy = list(np.load('avg_train_accuracy.npy'))
        total_vali_loss = list(np.load('total_vali_loss.npy'))
        total_vali_accuracy = list(np.load('total_vali_accuracy.npy'))
        epc_save = check_point['epoch'] - 1 
    
    tolerence = 3
    i = 0 
    for epoch in range(epc_save, num_epochs):
        N = 20
        M = 20
        N_minibatch_loss = 0.0    
        best_loss = 100
        early_stop = 0
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train']:
            scheduler.step()
            # Iterate over data.
            for minibatch_count, (inputs, labels) in enumerate(train_loader, 0):
                inputs = torch.squeeze(torch.stack([inputs,inputs,inputs], dim=1, out=None))
                inputs = inputs.to(computing_device)
                labels = labels.to(computing_device)
                labels = labels.long()
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    labels = labels.long()
                    loss = criterion(outputs,torch.max(labels, 1)[1])
                    #loss = criterion(torch.max(outputs,1)[1],torch.max(labels, 1)[1])
                    N_minibatch_loss += loss
                    # backward + optimize only if in training phase
                    loss.backward()
                    optimizer.step()
                
                # statistics
                # training stats
                if minibatch_count % N == 0 and minibatch_count!=0:    

                    # Print the loss averaged over the last N mini-batches    
                    N_minibatch_loss /= N
                    print('Epoch %d, average minibatch %d loss: %.3f' %
                        (epoch + 1, minibatch_count, N_minibatch_loss))

                    # Add the averaged loss over N minibatches and reset the counter
                    avg_minibatch_loss.append(N_minibatch_loss)
                    
                    avg_minibatch_loss_1 = np.array(avg_minibatch_loss)
                    np.save('avg_train_loss', avg_minibatch_loss_1)
                    
                    N_minibatch_loss = 0.0

                    output_np = outputs.cpu().detach().numpy()
                    label_np = labels.cpu().detach().numpy()

                    accuracy_train = accuracy(label_np, output_np)
                    avg_minibatch_accuracy.append(accuracy_train)
                    np.save('avg_train_accuracy', np.array(avg_minibatch_accuracy))
                    
                    print('accuracy',accuracy_train)
                    #print('accuracy, precision, recall', accuracy, precision, recall)
                
                #Validation
                if minibatch_count % M == 0 and minibatch_count!=0: 
                    #model = torch.load('./checkpoint')
                    save_checkpoint({'epoch': epoch + 1,
                                'state_dict': model.state_dict(),
                                'optimizer': optimizer.state_dict(),
                                },
                                filename='./checkpoint/'+'%d_model_epoch%d.pth' % (epoch , minibatch_count))
                    
                    v_loss, v_accuracy = validate(val_loader,model,optimizer)
#                     print(v_loss)
                    
                    # Save validation loss and accuracy
                    
                    total_vali_loss.append(v_loss)
                    total_vali_loss_1 = np.array(total_vali_loss)
                    np.save('total_vali_loss', total_vali_loss_1)                    
                    total_vali_accuracy.append(v_accuracy)
                    np.save('total_vali_accuracy', np.array(total_vali_accuracy))
                    
                    if total_vali_loss[i] > best_loss and i != 0:
                        early_stop += 1
                        if early_stop == tolerence:
                            print('early stop here')
                            break
                    else:
                        best_loss = total_vali_loss[i] 
                        early_stop = 0
                    i = i + 1
            print("Finished", epoch + 1, "epochs of training")
    print("Training complete after", epoch, "epochs")
    
    avg_minibatch_loss = np.array(avg_minibatch_loss)
    np.save('avg_minibatch_loss_new', avg_minibatch_loss)

    total_vali_loss = np.array(total_vali_loss)
    np.save('total_vali_loss_new', total_vali_loss)  
    print("total_vali_loss")
    print(total_vali_loss)
    print("avg_minibatch_loss")
    print(avg_minibatch_loss)
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s '.format(
        time_elapsed // 60, time_elapsed % 60))


In [14]:
def save_checkpoint(state, is_best=0, filename='models/checkpoint.pth.tar'):
    torch.save(state, filename)


In [15]:
model_trained = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=10, resume=True, direct="./checkpoint/5_model_epoch80.pth")


Resume model: ./checkpoint/5_model_epoch80.pth
Epoch 5/9
----------
Epoch 6, average minibatch 20 loss: 1.298
accuracy 0.609375
Vali Loss: 0.9511328935623169, Vali Accuracy: 0.7612898936170213
validation time =  21.439762353897095
[3.2833776473999023, 2.387113571166992, 2.0565390586853027, 1.9296038150787354, 1.794572353363037, 1.6674339771270752, 1.6271623373031616, 1.4941617250442505, 1.4461034536361694, 1.5105496644973755, 1.4769864082336426, 1.3600962162017822, 1.379114031791687, 1.426755666732788, 1.2887232303619385, 1.3071423768997192, 1.2464600801467896, 1.2198103666305542, 1.3316423892974854, 1.2218856811523438, 1.2348301410675049, 1.187071442604065, 1.1794453859329224, 1.2204697132110596, 1.2212944030761719, 1.2133207321166992, 1.190111756324768, 1.1349315643310547, 1.1595377922058105, 1.097761869430542, 1.143860101699829, 1.1460378170013428, 1.092395305633545, 1.2305432558059692, 1.0782526731491089, 1.1182498931884766, 1.1325337886810303, 1.0889010429382324, 1.193535804748535

Epoch 6, average minibatch 160 loss: 0.767
accuracy 0.71875
Vali Loss: 0.7497900724411011, Vali Accuracy: 0.7790159574468086
validation time =  22.578137397766113
[3.2833776473999023, 2.387113571166992, 2.0565390586853027, 1.9296038150787354, 1.794572353363037, 1.6674339771270752, 1.6271623373031616, 1.4941617250442505, 1.4461034536361694, 1.5105496644973755, 1.4769864082336426, 1.3600962162017822, 1.379114031791687, 1.426755666732788, 1.2887232303619385, 1.3071423768997192, 1.2464600801467896, 1.2198103666305542, 1.3316423892974854, 1.2218856811523438, 1.2348301410675049, 1.187071442604065, 1.1794453859329224, 1.2204697132110596, 1.2212944030761719, 1.2133207321166992, 1.190111756324768, 1.1349315643310547, 1.1595377922058105, 1.097761869430542, 1.143860101699829, 1.1460378170013428, 1.092395305633545, 1.2305432558059692, 1.0782526731491089, 1.1182498931884766, 1.1325337886810303, 1.0889010429382324, 1.1935358047485352, 1.1212092638015747, 1.1217806339263916, 1.1141881942749023, 1.113

Epoch 6, average minibatch 300 loss: 0.747
accuracy 0.765625
Vali Loss: 0.7830274701118469, Vali Accuracy: 0.7720079787234042
validation time =  25.26109528541565
[3.2833776473999023, 2.387113571166992, 2.0565390586853027, 1.9296038150787354, 1.794572353363037, 1.6674339771270752, 1.6271623373031616, 1.4941617250442505, 1.4461034536361694, 1.5105496644973755, 1.4769864082336426, 1.3600962162017822, 1.379114031791687, 1.426755666732788, 1.2887232303619385, 1.3071423768997192, 1.2464600801467896, 1.2198103666305542, 1.3316423892974854, 1.2218856811523438, 1.2348301410675049, 1.187071442604065, 1.1794453859329224, 1.2204697132110596, 1.2212944030761719, 1.2133207321166992, 1.190111756324768, 1.1349315643310547, 1.1595377922058105, 1.097761869430542, 1.143860101699829, 1.1460378170013428, 1.092395305633545, 1.2305432558059692, 1.0782526731491089, 1.1182498931884766, 1.1325337886810303, 1.0889010429382324, 1.1935358047485352, 1.1212092638015747, 1.1217806339263916, 1.1141881942749023, 1.113

Epoch 7, average minibatch 80 loss: 0.547
accuracy 0.765625
Vali Loss: 0.7571409344673157, Vali Accuracy: 0.7837898936170212
validation time =  23.71033763885498
[3.2833776473999023, 2.387113571166992, 2.0565390586853027, 1.9296038150787354, 1.794572353363037, 1.6674339771270752, 1.6271623373031616, 1.4941617250442505, 1.4461034536361694, 1.5105496644973755, 1.4769864082336426, 1.3600962162017822, 1.379114031791687, 1.426755666732788, 1.2887232303619385, 1.3071423768997192, 1.2464600801467896, 1.2198103666305542, 1.3316423892974854, 1.2218856811523438, 1.2348301410675049, 1.187071442604065, 1.1794453859329224, 1.2204697132110596, 1.2212944030761719, 1.2133207321166992, 1.190111756324768, 1.1349315643310547, 1.1595377922058105, 1.097761869430542, 1.143860101699829, 1.1460378170013428, 1.092395305633545, 1.2305432558059692, 1.0782526731491089, 1.1182498931884766, 1.1325337886810303, 1.0889010429382324, 1.1935358047485352, 1.1212092638015747, 1.1217806339263916, 1.1141881942749023, 1.1132

Epoch 8, average minibatch 120 loss: 0.563
accuracy 0.828125
Vali Loss: 0.8133112788200378, Vali Accuracy: 0.7742553191489362
validation time =  23.000022411346436
[3.2833776473999023, 2.387113571166992, 2.0565390586853027, 1.9296038150787354, 1.794572353363037, 1.6674339771270752, 1.6271623373031616, 1.4941617250442505, 1.4461034536361694, 1.5105496644973755, 1.4769864082336426, 1.3600962162017822, 1.379114031791687, 1.426755666732788, 1.2887232303619385, 1.3071423768997192, 1.2464600801467896, 1.2198103666305542, 1.3316423892974854, 1.2218856811523438, 1.2348301410675049, 1.187071442604065, 1.1794453859329224, 1.2204697132110596, 1.2212944030761719, 1.2133207321166992, 1.190111756324768, 1.1349315643310547, 1.1595377922058105, 1.097761869430542, 1.143860101699829, 1.1460378170013428, 1.092395305633545, 1.2305432558059692, 1.0782526731491089, 1.1182498931884766, 1.1325337886810303, 1.0889010429382324, 1.1935358047485352, 1.1212092638015747, 1.1217806339263916, 1.1141881942749023, 1.11

Epoch 9, average minibatch 100 loss: 0.567
accuracy 0.8125
Vali Loss: 0.7754183411598206, Vali Accuracy: 0.7806914893617022
validation time =  25.204617261886597
[3.2833776473999023, 2.387113571166992, 2.0565390586853027, 1.9296038150787354, 1.794572353363037, 1.6674339771270752, 1.6271623373031616, 1.4941617250442505, 1.4461034536361694, 1.5105496644973755, 1.4769864082336426, 1.3600962162017822, 1.379114031791687, 1.426755666732788, 1.2887232303619385, 1.3071423768997192, 1.2464600801467896, 1.2198103666305542, 1.3316423892974854, 1.2218856811523438, 1.2348301410675049, 1.187071442604065, 1.1794453859329224, 1.2204697132110596, 1.2212944030761719, 1.2133207321166992, 1.190111756324768, 1.1349315643310547, 1.1595377922058105, 1.097761869430542, 1.143860101699829, 1.1460378170013428, 1.092395305633545, 1.2305432558059692, 1.0782526731491089, 1.1182498931884766, 1.1325337886810303, 1.0889010429382324, 1.1935358047485352, 1.1212092638015747, 1.1217806339263916, 1.1141881942749023, 1.1132

Epoch 9, average minibatch 200 loss: 0.570
accuracy 0.890625
Vali Loss: 0.7978864312171936, Vali Accuracy: 0.7771409574468084
validation time =  24.46001696586609
[3.2833776473999023, 2.387113571166992, 2.0565390586853027, 1.9296038150787354, 1.794572353363037, 1.6674339771270752, 1.6271623373031616, 1.4941617250442505, 1.4461034536361694, 1.5105496644973755, 1.4769864082336426, 1.3600962162017822, 1.379114031791687, 1.426755666732788, 1.2887232303619385, 1.3071423768997192, 1.2464600801467896, 1.2198103666305542, 1.3316423892974854, 1.2218856811523438, 1.2348301410675049, 1.187071442604065, 1.1794453859329224, 1.2204697132110596, 1.2212944030761719, 1.2133207321166992, 1.190111756324768, 1.1349315643310547, 1.1595377922058105, 1.097761869430542, 1.143860101699829, 1.1460378170013428, 1.092395305633545, 1.2305432558059692, 1.0782526731491089, 1.1182498931884766, 1.1325337886810303, 1.0889010429382324, 1.1935358047485352, 1.1212092638015747, 1.1217806339263916, 1.1141881942749023, 1.113

Epoch 10, average minibatch 40 loss: 0.377
accuracy 0.890625
Vali Loss: 0.8138260245323181, Vali Accuracy: 0.7741090425531915
validation time =  25.666563034057617
[3.2833776473999023, 2.387113571166992, 2.0565390586853027, 1.9296038150787354, 1.794572353363037, 1.6674339771270752, 1.6271623373031616, 1.4941617250442505, 1.4461034536361694, 1.5105496644973755, 1.4769864082336426, 1.3600962162017822, 1.379114031791687, 1.426755666732788, 1.2887232303619385, 1.3071423768997192, 1.2464600801467896, 1.2198103666305542, 1.3316423892974854, 1.2218856811523438, 1.2348301410675049, 1.187071442604065, 1.1794453859329224, 1.2204697132110596, 1.2212944030761719, 1.2133207321166992, 1.190111756324768, 1.1349315643310547, 1.1595377922058105, 1.097761869430542, 1.143860101699829, 1.1460378170013428, 1.092395305633545, 1.2305432558059692, 1.0782526731491089, 1.1182498931884766, 1.1325337886810303, 1.0889010429382324, 1.1935358047485352, 1.1212092638015747, 1.1217806339263916, 1.1141881942749023, 1.11

In [17]:
np.load('./total_vali_accuracy.npy')

array([0.37117021, 0.45521277, 0.49990691, 0.51956117, 0.55      ,
       0.55228723, 0.5843883 , 0.59949468, 0.6225133 , 0.60651596,
       0.60917553, 0.62029255, 0.62191489, 0.61634309, 0.63776596,
       0.63610372, 0.65301862, 0.66406915, 0.64474734, 0.6687234 ,
       0.66547872, 0.6719016 , 0.67489362, 0.68257979, 0.67988032,
       0.66340426, 0.68135638, 0.69103723, 0.68117021, 0.69531915,
       0.68671543, 0.68523936, 0.69103723, 0.65305851, 0.70163564,
       0.68206117, 0.67902926, 0.6993883 , 0.68135638, 0.69143617,
       0.69808511, 0.70481383, 0.68179521, 0.68771277, 0.70946809,
       0.6975    , 0.71027926, 0.71452128, 0.76128989, 0.7612234 ,
       0.75966755, 0.77240691, 0.76801862, 0.74422872, 0.76      ,
       0.77901596, 0.77618351, 0.75035904, 0.77019947, 0.77485372,
       0.75324468, 0.76835106, 0.77200798, 0.7487367 , 0.76321809,
       0.76535904, 0.75804521, 0.77200798, 0.78378989, 0.76764628,
       0.78013298, 0.77639628, 0.74913564, 0.78696809, 0.77425